In [1]:
from __future__ import print_function
import os
import numpy as np
np.random.seed(1337)

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
import sys

Using TensorFlow backend.


In [2]:
import xml.etree.ElementTree as ET
print('Processing text dataset')

tree = ET.parse("./Restaurants.xml")
# tree = ET.parse("./Laptop.xml")
corpus = tree.getroot()
sentences = [] # List of list of sentences.
sent = corpus.findall('.//sentence')
for s in sent:
    sentences.append(s.find('text').text)

print ('Generated list of sentences..')
print(sentences)
print(len(sentences))  #3044

MAX_SEQ_LENGTH = 69
MAX_NB_WORDS = 40000
EMBEDDING_DIM = 300

Processing text dataset
Generated list of sentences..
['But the staff was so horrible to us.', "To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.", "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.", 'Where Gabriela personaly greets you and recommends you what to eat.', "For those that go once and don't enjoy it, all I can say is that they just don't get it.", "Not only was the food outstanding, but the little 'perks' were great.", 'It is very overpriced and not very tasty.', 'Our agreed favorite is the orrechiete with sausage and chicken (usually the waiters are kind enough to split the dish in half so you get to sample both meats).', 'The Bagels have an outstanding taste with a terrific texture, both chewy yet not gummy.', 'Nevertheless the food itself is pretty good.', "i've noti

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('./glove.6B.300d.txt',encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
# print(embeddings_index)
print(list(embeddings_index.items())[:2]) 
print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors.
[('the', array([ 4.6560e-02,  2.1318e-01, -7.4364e-03, -4.5854e-01, -3.5639e-02,
        2.3643e-01, -2.8836e-01,  2.1521e-01, -1.3486e-01, -1.6413e+00,
       -2.6091e-01,  3.2434e-02,  5.6621e-02, -4.3296e-02, -2.1672e-02,
        2.2476e-01, -7.5129e-02, -6.7018e-02, -1.4247e-01,  3.8825e-02,
       -1.8951e-01,  2.9977e-01,  3.9305e-01,  1.7887e-01, -1.7343e-01,
       -2.1178e-01,  2.3617e-01, -6.3681e-02, -4.2318e-01, -1.1661e-01,
        9.3754e-02,  1.7296e-01, -3.3073e-01,  4.9112e-01, -6.8995e-01,
       -9.2462e-02,  2.4742e-01, -1.7991e-01,  9.7908e-02,  8.3118e-02,
        1.5299e-01, -2.7276e-01, -3.8934e-02,  5.4453e-01,  5.3737e-01,
        2.9105e-01, -7.3514e-03,  4.7880e-02, -4.0760e-01, -2.6759e-02,
        1.7919e-01,  1.0977e-02, -1.0963e-01, -2.6395e-01,  7.3990e-02,
        2.6236e-01, -1.5080e-01,  3.4623e-01,  2.5758e-01,  1.1971e-01,
       -3.7135e-02, -7.1593e-02,  4.3898e-01, -4.0764e-02,  1.6425e-02,
       -4.4640e-01,  1.7197e-01,

In [4]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, lower=False)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(word_index)
data = pad_sequences(sequences, maxlen=MAX_SEQ_LENGTH, padding='post')
print(data)
print('Shape of data tensor:', data.shape)#2维数组，每一行代表一个句子，restaurants文件共有3044个句子

Found 5250 unique tokens.
{'the': 1, 'and': 2, 'a': 3, 'to': 4, 'is': 5, 'I': 6, 'was': 7, 'of': 8, 'for': 9, 'The': 10, 'in': 11, 'food': 12, 'it': 13, 'with': 14, 'you': 15, 'but': 16, 'place': 17, 'this': 18, 'that': 19, 'have': 20, 'not': 21, 'are': 22, 'on': 23, 'good': 24, 'at': 25, 'we': 26, 'great': 27, 'service': 28, 'were': 29, 'had': 30, 'be': 31, 'my': 32, 'restaurant': 33, 'they': 34, 'very': 35, 'so': 36, 'go': 37, 'as': 38, 'We': 39, 'there': 40, 'like': 41, 'been': 42, 'all': 43, 'here': 44, 'out': 45, 'from': 46, 'one': 47, 'an': 48, 'get': 49, 'would': 50, 'best': 51, 'or': 52, 'by': 53, 'This': 54, 'back': 55, 'your': 56, 'our': 57, 'if': 58, 'will': 59, 'more': 60, 'has': 61, 'time': 62, 'their': 63, 'menu': 64, 'about': 65, 'delicious': 66, 'always': 67, 'too': 68, 'than': 69, 'just': 70, 'only': 71, 'dinner': 72, 'well': 73, 'up': 74, 'can': 75, 'wine': 76, 'staff': 77, 'which': 78, "I've": 79, 'some': 80, "don't": 81, 'me': 82, 'excellent': 83, 'never': 84, 'nice

A:\anaconda\envs\condatest\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [5]:
# import nltk
# from keras.preprocessing.text import text_to_word_sequence
# raw_output = corpus.findall('.//sentence')
# train_out= np.zeros(shape=(3044,69))
# i=0
# for output in raw_output:
#     s = text_to_word_sequence(output.find('text').text, lower=False)
#     indices = np.zeros(MAX_SEQ_LENGTH)
    
#     aspectTerms = output.find('aspectTerms')
#     if (aspectTerms):
#         aspectTerm = aspectTerms.findall('aspectTerm')
#         if (aspectTerm):
#             for aspect_term in aspectTerm:
#                 try:
#                     indices[s.index(aspect_term.attrib['term'])] = 1
# #                     print (indices)
#                 except:
#                     continue
#     train_out[i] = indices
#     i=i+1

# print ("Shape of output tensor:", train_out.shape)
import nltk
from keras.preprocessing.text import text_to_word_sequence
raw_output = corpus.findall('.//sentence')
train_out = []
delet = []
print(data.shape)
data = np.array(data)
print(data.shape)
i=0
for output in raw_output:
    print('---')
    s = text_to_word_sequence(output.find('text').text, lower=True)
    print(s)
    indices = np.zeros(MAX_SEQ_LENGTH)
    
    aspectTerms = output.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        k=0
        if (len(aspectTerm)>0):
            for aspect_term in aspectTerm:
                try:
                    aspt = text_to_word_sequence(aspect_term.attrib['term'])
                    print(aspt)  #方面的列表：['orrechiete', 'with', 'sausage', 'and', 'chicken']
                    if(len(aspt) < 2):    #term长的不要
                        indices[s.index(aspt[0])] = 1
                    else:
                        k=1
                        break
                except:
                    continue
    else:
        k=1
    if(k==1):   #剔除长的，空的term
          delet.append(i)
    train_out.append(indices)
    i=i+1
print(train_out)
print(train_out[0].shape)
print(len(delet))   #准备删除没有方面的数据

(3044, 69)
(3044, 69)
---
['but', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us']
['staff']
---
['to', 'be', 'completely', 'fair', 'the', 'only', 'redeeming', 'factor', 'was', 'the', 'food', 'which', 'was', 'above', 'average', 'but', "couldn't", 'make', 'up', 'for', 'all', 'the', 'other', 'deficiencies', 'of', 'teodora']
['food']
---
['the', 'food', 'is', 'uniformly', 'exceptional', 'with', 'a', 'very', 'capable', 'kitchen', 'which', 'will', 'proudly', 'whip', 'up', 'whatever', 'you', 'feel', 'like', 'eating', 'whether', "it's", 'on', 'the', 'menu', 'or', 'not']
['food']
['kitchen']
['menu']
---
['where', 'gabriela', 'personaly', 'greets', 'you', 'and', 'recommends', 'you', 'what', 'to', 'eat']
---
['for', 'those', 'that', 'go', 'once', 'and', "don't", 'enjoy', 'it', 'all', 'i', 'can', 'say', 'is', 'that', 'they', 'just', "don't", 'get', 'it']
---
['not', 'only', 'was', 'the', 'food', 'outstanding', 'but', 'the', 'little', "'perks'", 'were', 'great']
['food']
['perks']
---
['it', 

['my', 'friend', 'from', 'milan', 'and', 'myself', 'were', 'pleasantly', 'surprised', 'when', 'we', 'arrived', 'and', 'everyone', 'spoke', 'italian']
---
['others', 'have', 'to', 'go', 'to', 'other', 'restaurants', 'and', 'feel', 'sad', 'until', "they're", 'eaten']
---
['why', 'make', 'a', 'reservation', 'if', 'you', "aren't", 'going', 'to', 'keep', 'it']
['reservation']
---
['the', 'pad', 'thai', 'is', 'excellent', 'here', 'as', 'well']
['pad', 'thai']
---
['big', 'and', 'gawky', 'on', 'the', 'outside']
---
['i', 'have', 'been', 'to', "mary's", 'many', 'times', 'and', 'i', 'like', 'it']
---
['searching', 'to', 'get', 'out', 'from', 'the', 'cold', 'freezing', 'weather', 'we', 'went', 'into', 'what', 'we', 'thought', 'was', 'a', 'deli']
---
['i', 'do', 'not', 'recommend', 'lunch', 'specials', 'just', 'because', 'it', 'tasts', 'the', 'same', 'with', 'other', 'regular', 'chinese', 'restaurant']
['lunch', 'specials']
---
['highly', 'recommended']
---
['this', 'is', 'the', 'best', 'thing', 

['roast', 'chicken']
---
['no', 'thanks']
---
['there', 'was', 'a', 'long', 'wait', 'for', 'a', 'table', 'outside', 'but', 'it', 'was', 'a', 'little', 'too', 'hot', 'in', 'the', 'sun', 'anyway', 'so', 'our', 'insde', 'table', 'was', 'very', 'nice']
['table']
['insde', 'table']
---
['i', 'would', 'definently', 'recommend', 'this', 'restaurant']
---
['the', 'prices', 'are', 'wonderfully', 'low']
['prices']
---
['if', 'you', 'like', 'spicy', 'food', 'get', 'the', 'chicken', 'vindaloo']
['chicken', 'vindaloo']
---
['i', 'love', 'this', 'place']
---
['i', 'have', 'never', 'been', 'disappointed', 'but', 'their', 'true', 'strength', 'lays', 'in', 'their', 'amazingly', 'delicious', 'and', 'cheap', 'lunch', 'specials']
['lunch', 'specials']
---
['so', 'much', 'more', 'than', 'the', 'usual', 'bar', 'food', 'go', 'there', 'to', 'enjoy', 'the', 'menu', 'while', 'sampling', 'one', 'of', 'their', 'hand', 'crafted', 'beers']
['bar', 'food']
---
['a', 'big', 'disappointment', 'all', 'around']
---
['an

['be', 'sure', 'to', 'try', 'the', 'seasonal', 'and', 'always', 'delicious', 'specials']
['specials']
---
['the', 'ambience', 'is', 'authentic', 'and', 'relaxing', 'and', 'we', 'have', 'always', 'received', 'attentive', 'and', 'prompt', 'service']
['ambience']
['service']
---
['fresh', 'authentic', 'french', 'cuisine', 'in', 'substantial', 'portions']
['french', 'cuisine']
---
['drinks', 'got', 'screwed', 'up', 'she', 'acted', 'put', 'upon']
['drinks']
---
['the', 'one', 'vegetarian', 'entree', "abby's", 'treasure', 'was', 'actually', 'quite', 'a', 'surprise', 'it', 'was', 'delicious', 'and', 'had', 'wintermelon', 'covering', 'an', 'assortment', 'of', 'fresh', 'mushrooms', 'and', 'vegetables']
['vegetarian', 'entree']
---
['at', '5', 'dumplings', 'for', '1', 'you', 'just', 'cannot', 'go', 'wrong']
['dumplings']
---
['but', 'the', 'pizza', 'is', 'way', 'to', 'expensive']
['pizza']
---
['and', 'the', 'staff', 'is', 'also', 'young', 'energeic', 'and', 'hot']
['staff']
---
['even', 'better

['the', 'only', 'concern', 'i', 'have', 'is', 'with', 'the', 'slighly', 'all', 'business', 'waitstaff', 'who', 'order', 'and', 'throw', 'the', 'food', 'down', 'rushing', 'you', 'out']
['waitstaff']
['food']
---
['have', 'the', 'iced', 'tea']
['iced', 'tea']
---
['for', 'park', 'avenue', 'i', 'expected', 'so', 'much', 'more']
---
['small', 'servings', 'for', 'main', 'entree', 'i', 'had', 'salmon', 'wasnt', 'impressed', 'girlfriend', 'had', 'chicken', 'it', 'was', 'good']
['salmon']
['chicken']
['servings']
['entree']
---
['so', 'rude']
---
['you', 'can', 'actually', 'get', '2', 'salads', 'worth', 'if', 'u', 'take', 'it', 'home', 'and', 'add', 'it', 'to', 'some', 'lettuce']
['salads']
['lettuce']
---
['the', 'ambience', 'is', 'very', 'romantic', 'and', 'definitely', 'a', 'good', 'place', 'to', 'bring', 'a', 'date']
['ambience']
['place']
---
['they', 'are', 'fast']
---
['it', 'is', 'far', 'more', 'popular', 'as', 'a', 'bar', 'than', 'as', 'a', 'restaurant', 'with', 'only', 'a', 'few', 't

---
['i', 'am', 'not', 'sure', 'if', 'i', 'would', 'call', 'the', 'food', 'here', 'indian', 'as', 'it', 'is', 'a', 'fusion', 'of', 'what', 'seems', 'to', 'be', 'french', 'with', 'an', 'indian', 'or', 'exotic', 'touch']
['food']
---
["i've", 'eaten', 'thai', 'many', 'times', 'and', 'am', 'very', 'familiar', 'with', 'the', 'cuisine']
['cuisine']
---
['the', 'waiter', 'was', 'attentive']
['waiter']
---
['their', 'margarita', 'is', 'best', "i've", 'had', 'since', "i've", 'returned', 'from', 'naples']
['margarita']
---
['i', 'highly', 'recommend', 'the', 'restaurant', 'based', 'on', 'our', 'experience', 'last', 'night']
---
['the', 'tastes', 'makes', 'your', 'mouth', 'water', 'for', 'more']
['tastes']
---
['i', 'came', 'into', 'the', 'city', 'to', 'try', 'out', 'this', 'restaurant', 'because', 'a', 'friend', 'had', 'recommended', 'it', 'to', 'my', 'husband']
---
['their', 'sake', 'list', 'was', 'extensive', 'but', 'we', 'were', 'looking', 'for', 'purple', 'haze', 'which', "wasn't", 'listed'

[array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [6]:
print('Preparing embedding matrix.')

# prepare embedding matrix
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, 300))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print(embedding_matrix.shape)

# load pre-trained word embeddings into an Embedding layer
# Here, we have set trainable = False so as to keep the embeddings fixed.
embedding_layer = Embedding(nb_words + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            trainable=False)
print('Embedding Layer set..')

Preparing embedding matrix.
(5251, 300)
Embedding Layer set..


In [7]:
from keras.models import Sequential
import keras.optimizers as opt
embedding_model = Sequential()
embedding_model.add(embedding_layer)
# optimizer = opt.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False, clipnorm=10, clipvalue=0)
embedding_model.compile(loss='categorical_crossentropy',
                        optimizer='RMSProp',
                        metrics=['acc']
                       )
embedding_output = embedding_model.predict(data)
print('Generated word Embeddings..')
print('Shape of Embedding_output', embedding_output.shape)

Generated word Embeddings..
Shape of Embedding_output (3044, 69, 300)


In [8]:
# from keras.preprocessing.text import text_to_word_sequence
# from nltk.tag.stanford import StanfordPOSTagger
# from sklearn import preprocessing
# from tqdm import tqdm

# train_input = np.zeros(shape=(3044,69,306))
# le = preprocessing.LabelEncoder()
# tags = ["CC","NN","JJ","VB","RB","IN"]
# le.fit(tags)
# i=0
# sentences = corpus.findall('.//sentence')
# for sent in sentences:
#     s = text_to_word_sequence(sent.find('text').text)
#     tags_for_sent = nltk.pos_tag(s)
#     sent_len = len(tags_for_sent)
#     ohe = [0]*6

#     for j in xrange(69):
#         if j< len(tags_for_sent) and tags_for_sent[j][1][:2] in tags:
#             ohe[le.transform(tags_for_sent[j][1][:2])] = 1
#         train_input[i][j] = np.concatenate([embedding_output[i][j],ohe])
#     i=i+1
    
# print('Concatenated Word-Embeddings and POS Tag Features..')
from keras.preprocessing.text import text_to_word_sequence
from nltk.tag.stanford import StanfordPOSTagger
from sklearn import preprocessing
from tqdm import tqdm

train_input = np.zeros(shape=(3044,69,306))
le = preprocessing.LabelEncoder()
tags = ["CC","NN","JJ","VB","RB","IN"]
le.fit(tags)
i=0
sentences = corpus.findall('.//sentence')
for sent in sentences:
    s = text_to_word_sequence(sent.find('text').text)  #每一个句子的分词结果
#     print(s)
    tags_for_sent = nltk.pos_tag(s) #词性标注
    print(tags_for_sent)
    sent_len = len(tags_for_sent)
    ohe = [0]*6
#     print(ohe)
        
    for j in range(69):
        list=[]
        if j< len(tags_for_sent) and tags_for_sent[j][1][:2] in tags:
            list.append(tags_for_sent[j][1][:2])
#             print(list)
            ohe[le.transform(list)[0]] = 1
        train_input[i][j] = np.concatenate([embedding_output[i][j],ohe]) #把词性追加在每个300维词向量的尾部，变成306维
    i=i+1
print(train_input[2][3])
print(len(train_out) )
j=1
for i in sorted(delet, reverse=True):
    j=j+1
    print(j)
    train_input = np.delete(train_input, (i), axis=0)
    train_out = np.delete(train_out, (i), axis=0)
print('Concatenated Word-Embeddings and POS Tag Features...')
print(len(train_input),len(train_input[0]),len(train_input[0][0]))
print(len(train_out),len(train_out[0]))
print(type(train_out),type(train_out[0]))
print(train_out)
print(train_input)



[('but', 'CC'), ('the', 'DT'), ('staff', 'NN'), ('was', 'VBD'), ('so', 'RB'), ('horrible', 'JJ'), ('to', 'TO'), ('us', 'PRP')]
[('to', 'TO'), ('be', 'VB'), ('completely', 'RB'), ('fair', 'JJ'), ('the', 'DT'), ('only', 'JJ'), ('redeeming', 'NN'), ('factor', 'NN'), ('was', 'VBD'), ('the', 'DT'), ('food', 'NN'), ('which', 'WDT'), ('was', 'VBD'), ('above', 'IN'), ('average', 'NN'), ('but', 'CC'), ("couldn't", 'NN'), ('make', 'VBP'), ('up', 'RP'), ('for', 'IN'), ('all', 'PDT'), ('the', 'DT'), ('other', 'JJ'), ('deficiencies', 'NNS'), ('of', 'IN'), ('teodora', 'NN')]
[('the', 'DT'), ('food', 'NN'), ('is', 'VBZ'), ('uniformly', 'JJ'), ('exceptional', 'JJ'), ('with', 'IN'), ('a', 'DT'), ('very', 'RB'), ('capable', 'JJ'), ('kitchen', 'NN'), ('which', 'WDT'), ('will', 'MD'), ('proudly', 'RB'), ('whip', 'VB'), ('up', 'RP'), ('whatever', 'WDT'), ('you', 'PRP'), ('feel', 'VBP'), ('like', 'IN'), ('eating', 'VBG'), ('whether', 'IN'), ("it's", 'NN'), ('on', 'IN'), ('the', 'DT'), ('menu', 'NN'), ('or',

[('the', 'DT'), ('fried', 'JJ'), ('rice', 'NN'), ('is', 'VBZ'), ('amazing', 'VBG'), ('here', 'RB')]
[('three', 'CD'), ('courses', 'NNS'), ('choices', 'NNS'), ('include', 'VBP'), ('excellent', 'JJ'), ('mussels', 'NNS'), ('puff', 'VBP'), ('pastry', 'NN'), ('goat', 'NN'), ('cheese', 'NN'), ('and', 'CC'), ('salad', 'NN'), ('with', 'IN'), ('a', 'DT'), ('delicious', 'JJ'), ('dressing', 'NN'), ('and', 'CC'), ('a', 'DT'), ('hanger', 'NN'), ('steak', 'NN'), ('au', 'NN'), ('poivre', 'NN'), ('that', 'WDT'), ('is', 'VBZ'), ('out', 'IN'), ('of', 'IN'), ('this', 'DT'), ('world', 'NN')]
[("it's", 'VB'), ('a', 'DT'), ('perfect', 'JJ'), ('place', 'NN'), ('to', 'TO'), ('have', 'VB'), ('a', 'DT'), ('amanzing', 'VBG'), ('indian', 'JJ'), ('food', 'NN')]
[('the', 'DT'), ('place', 'NN'), ('is', 'VBZ'), ('so', 'RB'), ('cool', 'JJ'), ('and', 'CC'), ('the', 'DT'), ('service', 'NN'), ('is', 'VBZ'), ('prompt', 'JJ'), ('and', 'CC'), ('curtious', 'JJ')]
[('at', 'IN'), ('the', 'DT'), ('end', 'NN'), ("you're", 'NN'),

[('the', 'DT'), ('cuisine', 'NN'), ('from', 'IN'), ('what', 'WP'), ("i've", 'NN'), ('gathered', 'VBN'), ('is', 'VBZ'), ('authentic', 'JJ'), ('taiwanese', 'JJ'), ('though', 'IN'), ('its', 'PRP$'), ('very', 'RB'), ('different', 'JJ'), ('from', 'IN'), ('what', 'WP'), ("i've", 'VBP'), ('been', 'VBN'), ('accustomed', 'VBN'), ('to', 'TO'), ('in', 'IN'), ('taipei', 'NN')]
[('i', 'JJ'), ('heartily', 'RB'), ('recommend', 'VBP')]
[('heartland', 'NN'), ('brewery', 'NN'), ('is', 'VBZ'), ('by', 'IN'), ('far', 'RB'), ('one', 'CD'), ('of', 'IN'), ('the', 'DT'), ('best', 'JJS'), ('places', 'NNS'), ('in', 'IN'), ('nyc', 'NN'), ('to', 'TO'), ('make', 'VB'), ('new', 'JJ'), ('friends', 'NNS'), ('and', 'CC'), ('catch', 'VB'), ('up', 'RP'), ('with', 'IN'), ('old', 'JJ'), ('ones', 'NNS')]
[('i', 'NN'), ('almost', 'RB'), ('hesititate', 'NN'), ('to', 'TO'), ('write', 'VB'), ('a', 'DT'), ('review', 'NN'), ('because', 'IN'), ('the', 'DT'), ('atmosphere', 'NN'), ('was', 'VBD'), ('so', 'RB'), ('great', 'JJ'), ('an

[('good', 'JJ'), ('selection', 'NN'), ('of', 'IN'), ('wines', 'NNS'), ('ranging', 'VBG'), ('from', 'IN'), ('affordable', 'JJ'), ('to', 'TO'), ('high', 'JJ'), ('end', 'NN')]
[('nice', 'JJ'), ('restaurant', 'NN'), ('overall', 'JJ'), ('with', 'IN'), ('classic', 'JJ'), ('upscale', 'JJ'), ('italian', 'JJ'), ('decor', 'NN')]
[('not', 'RB'), ('impressed', 'VBN'), ('with', 'IN'), ('the', 'DT'), ('food', 'NN')]
[('the', 'DT'), ('entire', 'JJ'), ('dining', 'NN'), ('experience', 'NN'), ('was', 'VBD'), ('wonderful', 'JJ')]
[('the', 'DT'), ('wine', 'NN'), ('selection', 'NN'), ('by', 'IN'), ('the', 'DT'), ('glass', 'NN'), ('and', 'CC'), ('bottle', 'NN'), ('is', 'VBZ'), ('wonderful', 'JJ'), ('and', 'CC'), ('i', 'JJ'), ('always', 'RB'), ('recommend', 'VBP'), ('that', 'IN'), ('friends', 'NNS'), ('make', 'VBP'), ('a', 'DT'), ('reservation', 'NN'), ('if', 'IN'), ("they're", 'NN'), ('going', 'VBG'), ('to', 'TO'), ('be', 'VB'), ('in', 'IN'), ('town', 'NN')]
[('service', 'NN'), ('is', 'VBZ'), ('highly', 'RB

[('look', 'NN'), ('i', 'JJ'), ('dont', 'NN'), ('know', 'VBP'), ('if', 'IN'), ('the', 'DT'), ('people', 'NNS'), ('who', 'WP'), ('wrote', 'VBD'), ('the', 'DT'), ('reviews', 'NNS'), ('have', 'VBP'), ('ever', 'RB'), ('been', 'VBN'), ('to', 'TO'), ('a', 'DT'), ('decent', 'JJ'), ('place', 'NN'), ('please', 'NN'), ('try', 'VB'), ('someother', 'JJR'), ('places', 'NNS')]
[('suan', 'NN'), ('has', 'VBZ'), ('challenged', 'VBN'), ('this', 'DT'), ('loyalty', 'NN')]
[('our', 'PRP$'), ('favorite', 'JJ'), ('meal', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('pesto', 'JJ'), ('pizza', 'NN'), ('the', 'DT'), ('house', 'NN'), ('salad', 'NN'), ('and', 'CC'), ('a', 'DT'), ('good', 'JJ'), ('bottle', 'NN'), ('of', 'IN'), ('wine', 'NN')]
[('and', 'CC'), ('kruno', 'VB'), ('the', 'DT'), ('beverage', 'NN'), ('manager', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('best', 'JJS'), ('bartender', 'NN'), ('i', 'NN'), ('have', 'VBP'), ('yet', 'RB'), ('to', 'TO'), ('come', 'VB'), ('across', 'IN')]
[('ambiance', 'NN'), ('and', 'CC'), ('m

...

[('brava', 'NN'), ('la', 'NN'), ('pizza', 'NN'), ('fresca', 'NN')]
[('if', 'IN'), ('you', 'PRP'), ('are', 'VBP'), ('someone', 'NN'), ('who', 'WP'), ('appreciates', 'VBZ'), ('simplicity', 'NN'), ('elegance', 'NN'), ('and', 'CC'), ('wonderfully', 'RB'), ('presented', 'VBN'), ('and', 'CC'), ('tasting', 'VBG'), ('seafood', 'NN'), ('and', 'CC'), ('vegetables', 'NNS'), ('regardless', 'RB'), ('of', 'IN'), ('portion', 'NN'), ('size', 'NN'), ('kai', 'NN'), ('is', 'VBZ'), ('your', 'PRP$'), ('place', 'NN')]
[('the', 'DT'), ('prices', 'NNS'), ('cannot', 'VBP'), ('be', 'VB'), ('beat', 'VBN'), ('for', 'IN'), ('the', 'DT'), ('quality', 'NN'), ('quantity', 'NN'), ('freshness', 'NN'), ('and', 'CC'), ('taste', 'NN')]
[('check', 'VB'), ('it', 'PRP'), ('out', 'IN'), ('it', 'PRP'), ("won't", 'VBZ'), ('hurt', 'VB'), ('your', 'PRP$'), ('stomach', 'NN'), ('or', 'CC'), ('your', 'PRP$'), ('wallet', 'NN')]
[('if', 'IN'), ('just', 'RB'), ('to', 'TO'), ('drop', 'VB'), ('in', 'IN'), ('quickly', 'RB'), ('stuff', 'VB

[("i'd", 'NN'), ('highly', 'RB'), ('recommend', 'VB'), ('it', 'PRP'), ('for', 'IN'), ('a', 'DT'), ('special', 'JJ'), ('occasion', 'NN'), ('it', 'PRP'), ('provides', 'VBZ'), ('and', 'CC'), ('intimate', 'VB'), ('setting', 'NN'), ('and', 'CC'), ('nice', 'JJ'), ('service', 'NN')]
[('the', 'DT'), ('burger', 'NN'), ('was', 'VBD'), ('great', 'JJ'), ('also', 'RB')]
[('they', 'PRP'), ('did', 'VBD'), ('give', 'VB'), ('a', 'DT'), ('15', 'CD'), ('discount', 'NN'), ('at', 'IN'), ('the', 'DT'), ('end', 'NN'), ("wasn't", 'NN'), ('enough', 'JJ'), ('as', 'IN'), ('they', 'PRP'), ('knew', 'VBD'), ('the', 'DT'), ('service', 'NN'), ('was', 'VBD'), ('horrible', 'JJ')]
[('just', 'RB'), ('because', 'IN'), ("it's", 'JJ'), ('cheap', 'NN'), ('does', 'VBZ'), ('not', 'RB'), ('mean', 'VB'), ('the', 'DT'), ('portions', 'NNS'), ('are', 'VBP'), ('small', 'JJ'), ('or', 'CC'), ('the', 'DT'), ('food', 'NN'), ('is', 'VBZ'), ('nasty', 'JJ'), ('it', 'PRP'), ('is', 'VBZ'), ('great', 'JJ')]
[('the', 'DT'), ('pizza', 'NN'), ('

[('i', 'JJ'), ('went', 'VBD'), ('to', 'TO'), ('amma', 'VB'), ('after', 'IN'), ('a', 'DT'), ('friend', 'NN'), ('recommended', 'VBD'), ('it', 'PRP')]
[("i'm", 'NN'), ('just', 'RB'), ('reading', 'VBG'), ('some', 'DT'), ('of', 'IN'), ('these', 'DT'), ('reviews', 'NNS'), ('and', 'CC'), ('its', 'PRP$'), ('nonsense', 'NN'), ('that', 'IN'), ('people', 'NNS'), ('are', 'VBP'), ('reviewing', 'VBG'), ('some', 'DT'), ('places', 'NNS'), ('because', 'IN'), ('it', 'PRP'), ('was', 'VBD'), ('too', 'RB'), ('loud', 'JJ')]
[('kind', 'NN'), ('of', 'IN'), ('a', 'DT'), ('small', 'JJ'), ('place', 'NN'), ('but', 'CC'), ('i', 'NN'), ('guess', 'VBP'), ('if', 'IN'), ('they', 'PRP'), ('are', 'VBP'), ('not', 'RB'), ('too', 'RB'), ('busy', 'JJ'), ('might', 'MD'), ('be', 'VB'), ('able', 'JJ'), ('to', 'TO'), ('fit', 'VB'), ('a', 'DT'), ('group', 'NN'), ('or', 'CC'), ('kids', 'NNS')]
[('maybe', 'RB'), ('tomorrow', 'NN')]
[('we', 'PRP'), ('also', 'RB'), ('ordered', 'VBD'), ('two', 'CD'), ('hot', 'JJ'), ('dogs', 'NNS'), (

[('want', 'NN'), ('to', 'TO'), ('impress', 'VB')]
[('the', 'DT'), ('selection', 'NN'), ('changes', 'NNS'), ('frequently', 'RB'), ('but', 'CC'), ('the', 'DT'), ('basic', 'JJ'), ('dishes', 'NNS'), ('are', 'VBP'), ('always', 'RB'), ('available', 'JJ')]
[("we've", 'NN'), ('lived', 'VBN'), ('in', 'IN'), ('the', 'DT'), ('area', 'NN'), ('for', 'IN'), ('more', 'JJR'), ('than', 'IN'), ('8', 'CD'), ('years', 'NNS')]
[('get', 'VB'), ('the', 'DT'), ('soup', 'NN'), ('and', 'CC'), ('a', 'DT'), ('nosh', 'JJ'), ('pastrami', 'NN'), ('sandwich', 'NN'), ('for', 'IN'), ('8', 'CD'), ('and', 'CC'), ("you're", 'JJR'), ('golden', 'JJ')]
[('i', 'NNS'), ('have', 'VBP'), ('to', 'TO'), ('say', 'VB'), ('that', 'IN'), ('if', 'IN'), ('this', 'DT'), ('what', 'WDT'), ('makes', 'VBZ'), ('it', 'PRP'), ('easier', 'JJR'), ('to', 'TO'), ('get', 'VB'), ('a', 'DT'), ('saet', 'NN'), ('a', 'DT'), ('lunch', 'NN'), ('i', 'NN'), ('dont', 'VBP'), ('mind', 'NN')]
[('my', 'PRP$'), ('family', 'NN'), ('and', 'CC'), ('i', 'NN'), ('ate'

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


In [9]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution1D, Convolution2D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.preprocessing import sequence
from keras.optimizers import *
from keras.regularizers import l2

In [10]:
import keras.optimizers as opt
print('Training Model...')
model = Sequential()
# model.add(Convolution1D(100, 5, border_mode="same", input_shape=(69, 306)))
model.add(Convolution1D(100, 2, border_mode="same", input_shape=(69, 306)))
model.add(Activation("tanh"))
# model.add(MaxPooling1D(pool_length=5))
model.add(MaxPooling1D(pool_length=2))
model.add(Convolution1D(50, 3, border_mode="same"))
model.add(Activation("tanh"))
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("tanh"))
# softmax classifier
model.add(Dense(69, W_regularizer=l2(0.01)))
model.add(Activation("softmax"))

# model.load_weights('aspect_model_wepos.h5')
# model.compile(loss='categorical_crossentropy',
model.compile(loss='mean_squared_error',
#               optimizer=optimizer,
              optimizer='sgd',
              metrics=['acc'])

print('Model Trained.')

Training Model...
Model Trained.

A:\anaconda\envs\condatest\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(100, 2, input_shape=(69, 306), padding="same")`
  """
A:\anaconda\envs\condatest\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  
A:\anaconda\envs\condatest\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(50, 3, padding="same")`
  if __name__ == '__main__':
A:\anaconda\envs\condatest\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  # This is added back by InteractiveShellApp.init_path()
A:\anaconda\envs\condatest\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(69, kernel_regularizer=<keras.reg...)`
  app.launch_new_instance()


In [11]:
model.fit(train_input, train_out,
          validation_split=0.1,
          batch_size=100,
          nb_epoch=60
         )

111111


A:\anaconda\envs\condatest\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


222222
Tensor("Mean_1:0", shape=(), dtype=float32)
(2, 306, 100) (100,)
[<tf.Tensor 'gradients/conv1d_1/convolution/ExpandDims_1_grad/Reshape:0' shape=(2, 306, 100) dtype=float32>, <tf.Tensor 'gradients/conv1d_1/Reshape_grad/Reshape:0' shape=(100,) dtype=float32>, <tf.Tensor 'gradients/conv1d_2/convolution/ExpandDims_1_grad/Reshape:0' shape=(3, 100, 50) dtype=float32>, <tf.Tensor 'gradients/conv1d_2/Reshape_grad/Reshape:0' shape=(50,) dtype=float32>, <tf.Tensor 'gradients/dense_1/MatMul_grad/MatMul_1:0' shape=(850, 500) dtype=float32>, <tf.Tensor 'gradients/dense_1/BiasAdd_grad/BiasAddGrad:0' shape=(500,) dtype=float32>, <tf.Tensor 'gradients/AddN:0' shape=(500, 69) dtype=float32>, <tf.Tensor 'gradients/dense_2/BiasAdd_grad/BiasAddGrad:0' shape=(69,) dtype=float32>]
Train on 1170 samples, validate on 131 samples
Epoch 1/60
1170/1170 [==============================] - 1s 920us/step - loss: 1.2257 - acc: 0.0077 - val_loss: 1.2216 - val_acc: 0.0000e+00
Epoch 2/60
1170/1170 [==============

1170/1170 [==============================] - 1s 671us/step - loss: 0.9600 - acc: 0.0094 - val_loss: 0.9566 - val_acc: 0.0000e+00
Epoch 54/60
1170/1170 [==============================] - 1s 761us/step - loss: 0.9555 - acc: 0.0094 - val_loss: 0.9521 - val_acc: 0.0000e+00
Epoch 55/60
1170/1170 [==============================] - 1s 773us/step - loss: 0.9510 - acc: 0.0094 - val_loss: 0.9476 - val_acc: 0.0000e+00
Epoch 56/60
1170/1170 [==============================] - 1s 723us/step - loss: 0.9466 - acc: 0.0103 - val_loss: 0.9432 - val_acc: 0.0000e+00
Epoch 57/60
1170/1170 [==============================] - 1s 700us/step - loss: 0.9422 - acc: 0.0103 - val_loss: 0.9388 - val_acc: 0.0000e+00
Epoch 58/60
1170/1170 [==============================] - 1s 724us/step - loss: 0.9378 - acc: 0.0103 - val_loss: 0.9344 - val_acc: 0.0000e+00
Epoch 59/60
1170/1170 [==============================] - 1s 688us/step - loss: 0.9334 - acc: 0.0103 - val_loss: 0.9300 - val_acc: 0.0000e+00
Epoch 60/60
1170/1170 [==

In [12]:
# model.save_weights('aspect_wepos.h5')
model.save('m1.h5')  #   https://blog.csdn.net/leviopku/article/details/86612293
y_pred = model.predict(train_input[:1301])
# print(train_input[:2739])
print(y_pred)
print(len(y_pred))  #删除后余1301条数据
print(len(y_pred[0])) 

[[0.01898362 0.01376326 0.01450253 ... 0.02209694 0.01484107 0.00767617]
 [0.02033811 0.01063581 0.01202911 ... 0.02120746 0.01413212 0.01051342]
 [0.01487373 0.013675   0.01620491 ... 0.01995749 0.01312057 0.00764315]
 ...
 [0.0157292  0.01251866 0.01345881 ... 0.0168757  0.01323123 0.00978533]
 [0.01315597 0.01072329 0.01141302 ... 0.01681938 0.01498905 0.0088754 ]
 [0.01628573 0.01406935 0.01317952 ... 0.01927016 0.01550468 0.01099443]]
1301
69


In [13]:
processed_output = []
for i in range(len(y_pred)):
    processed_label =[]
    for j in range(len(y_pred[i])):
        if y_pred[i][j] > 0.27:
            processed_label.append(1)
        else:
            processed_label.append(0)
    processed_output.append(processed_label)
print(processed_output)
print(len(processed_output))
print(len(processed_output[0]))

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
def lookup(tokenizer, vec, returnIntNotWord=True):
    twordkey = [(k, tokenizer.word_index[k]) for k in sorted(tokenizer.word_index, key=tokenizer.word_index.get, reverse=False)]
    oneHotVec = [] #captures the index of the ords
    engVec = [] #this one returns the indexs and the words. Make sure returnIntNotWord is false though
    for eachRow, notUsed in enumerate(vec):
        for index, item in enumerate(vec[0]):
            if vec[eachRow][index] == 1:
                oneHotVec.append(index)
    for index in oneHotVec:
        engVec.append(twordkey[index])
    if returnIntNotWord == True:
        return oneHotVec
    else:
        return engVec
test_data = train_out[:1301]
total_pos = 0.0
true_pos = 0.0
total_neg = 0.0
true_neg = 0.0
for i in range(len(test_data)):
    for j in range(len(test_data[i])):
        if test_data[i][j] == 1:
            total_pos += 1
            if processed_output[i][j] ==1:
                
                true_pos +=1
            if processed_output[i][j] == 0:
                pass
#                 print(lookup(tokenizer,test_data[i],True))
        if test_data[i][j] == 0:
            total_neg += 1
            if processed_output[i][j] ==0:
                true_neg += 1

false_pos = total_neg-true_neg
false_neg = total_pos-true_pos
print(true_pos,true_neg,false_pos,false_neg)

0.0 87686.0 0.0 2083.0


In [15]:
precision = true_pos/(true_pos+false_pos)
recall = true_pos/total_pos
f1_score = 2*precision*recall/(precision+recall)
print ("precision - " +str(precision) + ", recall- " +str(recall)+ ", f1_score- " +str(f1_score))

ZeroDivisionError: float division by zero